In [1]:
import sys
import os

import argparse
import requests
from pprint import pprint
import numpy as np

sys.path.append("../")
import predictlol as pl
import config

api_key = config.api_key

username = 'delphinus6'
region = 'na1'
# match_index = 0

summoner_name, summoner_id, account_id = pl.get_summoner_ids(username, api_key, region)


In [2]:
matchlist = []
start_index = 0

response = requests.get("https://%s.api.riotgames.com/lol/match/v3/matchlists/by-account/%s?beginIndex=%s&api_key=%s" % (region, account_id, start_index, api_key))
matchlist_packet = response.json()

while len(matchlist_packet['matches']) > 0:
    matchlist.extend(matchlist_packet['matches'])
    start_index += 100
    response = requests.get("https://%s.api.riotgames.com/lol/match/v3/matchlists/by-account/%s?beginIndex=%s&api_key=%s" % (region, account_id, start_index, api_key))
    matchlist_packet = response.json()

In [3]:
matchlist[0]

{'platformId': 'NA1',
 'gameId': 2832713485,
 'champion': 11,
 'queue': 470,
 'season': 11,
 'timestamp': 1532495938761,
 'role': 'NONE',
 'lane': 'JUNGLE'}

In [25]:
for key in matchlist[0].keys():
    print(key)

platformId
gameId
champion
queue
season
timestamp
role
lane


In [6]:
match_data = pl.get_match_by_id(matchlist[0]['gameId'], api_key)
match_data

{'gameId': 2832713485,
 'platformId': 'NA1',
 'gameCreation': 1532495938761,
 'gameDuration': 1298,
 'queueId': 470,
 'mapId': 10,
 'seasonId': 11,
 'gameVersion': '8.14.238.9731',
 'gameMode': 'CLASSIC',
 'gameType': 'MATCHED_GAME',
 'teams': [{'teamId': 100,
   'win': 'Fail',
   'firstBlood': True,
   'firstTower': False,
   'firstInhibitor': False,
   'firstBaron': False,
   'firstDragon': False,
   'firstRiftHerald': False,
   'towerKills': 3,
   'inhibitorKills': 0,
   'baronKills': 0,
   'dragonKills': 0,
   'vilemawKills': 0,
   'riftHeraldKills': 0,
   'dominionVictoryScore': 0,
   'bans': [{'championId': 85, 'pickTurn': 1},
    {'championId': 31, 'pickTurn': 3},
    {'championId': 86, 'pickTurn': 5}]},
  {'teamId': 200,
   'win': 'Win',
   'firstBlood': False,
   'firstTower': True,
   'firstInhibitor': True,
   'firstBaron': False,
   'firstDragon': False,
   'firstRiftHerald': False,
   'towerKills': 7,
   'inhibitorKills': 2,
   'baronKills': 0,
   'dragonKills': 0,
   'vil

In [7]:
match_data['teams'][0]['win'] 

'Fail'

In [16]:
for participant in match_data['participantIdentities']:
    if participant['player']['summonerName'] == summoner_name:
        participant_id = participant['participantId']

stats = match_data['participants'][int(participant_id) - 1]['stats']
win = stats['win']
kills = stats['kills']
deaths = stats['deaths']
assists = stats['assists']

stats

{'participantId': 6,
 'win': True,
 'item0': 1419,
 'item1': 3124,
 'item2': 3144,
 'item3': 3006,
 'item4': 0,
 'item5': 0,
 'item6': 3348,
 'kills': 5,
 'deaths': 8,
 'assists': 8,
 'largestKillingSpree': 2,
 'largestMultiKill': 2,
 'killingSprees': 1,
 'longestTimeSpentLiving': 274,
 'doubleKills': 1,
 'tripleKills': 0,
 'quadraKills': 0,
 'pentaKills': 0,
 'unrealKills': 0,
 'totalDamageDealt': 128707,
 'magicDamageDealt': 3751,
 'physicalDamageDealt': 107562,
 'trueDamageDealt': 17393,
 'largestCriticalStrike': 0,
 'totalDamageDealtToChampions': 17885,
 'magicDamageDealtToChampions': 233,
 'physicalDamageDealtToChampions': 9911,
 'trueDamageDealtToChampions': 7739,
 'totalHeal': 9493,
 'totalUnitsHealed': 1,
 'damageSelfMitigated': 12128,
 'damageDealtToObjectives': 14698,
 'damageDealtToTurrets': 6644,
 'visionScore': 0,
 'timeCCingOthers': 0,
 'totalDamageTaken': 24449,
 'magicalDamageTaken': 6178,
 'physicalDamageTaken': 17135,
 'trueDamageTaken': 1136,
 'goldEarned': 10812,
 '

In [47]:
import json
import time
start = time.time()

wins = 0
total = 0
index = 0
all_stats = []
for match in matchlist:
    match_data = pl.get_match_by_id(match['gameId'], api_key)
    for participant in match_data['participantIdentities']:
        if participant['player']['summonerName'] == summoner_name:
            participant_id = participant['participantId']
    time.sleep(2)

    stats = match_data['participants'][int(participant_id) - 1]['stats']
    for key in match.keys():
        stats[key] = match[key]
    all_stats.append(stats)
    if stats['win']:
        wins += 1
    total += 1
    
    now = time.time()
    delay = (now - start)
    h = int(np.floor(delay / 3600.))
    m = int(np.floor((delay - 3600. * h) / 60.))
    s = (delay - 3600. * h - 60. * m)
    print('Game %s, t = %02d:%02d:%2.3f' % (total, h, m, s))
    
with open('all_stats.json', 'w') as fn:
    json.dump(all_stats, fn)
    
print(wins, total, wins/total)

Game 1, t = 00:00:2.601
Game 2, t = 00:00:5.521
Game 3, t = 00:00:7.679
Game 4, t = 00:00:9.814
Game 5, t = 00:00:11.946
Game 6, t = 00:00:14.073
Game 7, t = 00:00:16.224
Game 8, t = 00:00:18.431
Game 9, t = 00:00:20.624
Game 10, t = 00:00:22.804
Game 11, t = 00:00:24.930
Game 12, t = 00:00:27.074
Game 13, t = 00:00:29.279
Game 14, t = 00:00:31.418
Game 15, t = 00:00:33.557
Game 16, t = 00:00:36.374
Game 17, t = 00:00:38.713
Game 18, t = 00:00:40.846
Game 19, t = 00:00:43.081
Game 20, t = 00:00:45.378
Game 21, t = 00:00:47.712
Game 22, t = 00:00:49.853
Game 23, t = 00:00:52.012
Game 24, t = 00:00:54.165
Game 25, t = 00:00:56.305
Game 26, t = 00:00:58.767
Game 27, t = 00:01:0.939
Game 28, t = 00:01:3.115
Game 29, t = 00:01:5.251
Game 30, t = 00:01:7.397
Game 31, t = 00:01:9.679
Game 32, t = 00:01:11.816
Game 33, t = 00:01:14.646
Game 34, t = 00:01:16.806
Game 35, t = 00:01:19.028
Game 36, t = 00:01:21.185
Game 37, t = 00:01:23.328
Game 38, t = 00:01:25.454
Game 39, t = 00:01:27.912
Game

In [28]:
a = time.time()


In [29]:
b = time.time()
b-a

3.284541130065918

In [40]:
'%.02d' % 5

'05'

In [2]:
try:
    import simplejson as json
except ImportError:
    import json

soloq = []
with open('all_stats.json', 'r') as fn:
    all_stats = json.load(fn)
    
index = 0
for stat in all_stats:
    if stat['queue'] == 420:
        soloq.append(stat)
        index += 1
index

649

In [3]:
wins = 0
total = 0
for game in soloq[0:100]:
    if game['win']:
        wins += 1
    total += 1
print(wins, total, wins/total*100)

49 100 49.0


In [4]:
# Save by account id
account_id

234507298

In [54]:
all_stats[0]

{'participantId': 6,
 'win': True,
 'item0': 1419,
 'item1': 3124,
 'item2': 3144,
 'item3': 3006,
 'item4': 0,
 'item5': 0,
 'item6': 3348,
 'kills': 5,
 'deaths': 8,
 'assists': 8,
 'largestKillingSpree': 2,
 'largestMultiKill': 2,
 'killingSprees': 1,
 'longestTimeSpentLiving': 274,
 'doubleKills': 1,
 'tripleKills': 0,
 'quadraKills': 0,
 'pentaKills': 0,
 'unrealKills': 0,
 'totalDamageDealt': 128707,
 'magicDamageDealt': 3751,
 'physicalDamageDealt': 107562,
 'trueDamageDealt': 17393,
 'largestCriticalStrike': 0,
 'totalDamageDealtToChampions': 17885,
 'magicDamageDealtToChampions': 233,
 'physicalDamageDealtToChampions': 9911,
 'trueDamageDealtToChampions': 7739,
 'totalHeal': 9493,
 'totalUnitsHealed': 1,
 'damageSelfMitigated': 12128,
 'damageDealtToObjectives': 14698,
 'damageDealtToTurrets': 6644,
 'visionScore': 0,
 'timeCCingOthers': 0,
 'totalDamageTaken': 24449,
 'magicalDamageTaken': 6178,
 'physicalDamageTaken': 17135,
 'trueDamageTaken': 1136,
 'goldEarned': 10812,
 '

In [22]:
soloq[0]

{'participantId': 8,
 'win': True,
 'item0': 1419,
 'item1': 3022,
 'item2': 3153,
 'item3': 3124,
 'item4': 3047,
 'item5': 3211,
 'item6': 3340,
 'kills': 15,
 'deaths': 7,
 'assists': 3,
 'largestKillingSpree': 8,
 'largestMultiKill': 4,
 'killingSprees': 3,
 'longestTimeSpentLiving': 913,
 'doubleKills': 2,
 'tripleKills': 2,
 'quadraKills': 1,
 'pentaKills': 0,
 'unrealKills': 0,
 'totalDamageDealt': 182613,
 'magicDamageDealt': 4198,
 'physicalDamageDealt': 154716,
 'trueDamageDealt': 23698,
 'largestCriticalStrike': 0,
 'totalDamageDealtToChampions': 20167,
 'magicDamageDealtToChampions': 286,
 'physicalDamageDealtToChampions': 13671,
 'trueDamageDealtToChampions': 6210,
 'totalHeal': 9504,
 'totalUnitsHealed': 1,
 'damageSelfMitigated': 15460,
 'damageDealtToObjectives': 33466,
 'damageDealtToTurrets': 5867,
 'visionScore': 19,
 'timeCCingOthers': 2,
 'totalDamageTaken': 28705,
 'magicalDamageTaken': 7131,
 'physicalDamageTaken': 21433,
 'trueDamageTaken': 141,
 'goldEarned': 1

In [52]:
import json
import csv

with open('all_stats.json', 'r') as fn:
    all_stats = json.load(fn)

# with open('all_stats.csv', 'w') as fn:
#     csvwriter = csv.writer(fn)
#     header = all_stats[0].keys()
#     csvwriter.writerow(header)
#     for game in all_stats:
#         csvwriter.writerow(game.values())

with open('all_stats.csv', 'w') as fn:
    header = []
    for game in all_stats:
        header = list(set().union(header, list(game.keys())))
    writer = csv.DictWriter(fn, fieldnames=header)
    writer.writeheader()
    for game in all_stats:
        writer.writerow(game)

In [5]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017')

In [14]:
db = client['predictlol']

In [13]:
# client.drop_database('1')
client.list_database_names()

['admin', 'config', 'local', 'pymongo_test']

In [10]:
posts = db.posts
post_data = {
    'title': 'Python and MongoDB',
    'content': 'PyMongo is fun, you guys',
    'author': 'Scott'
}
result = posts.insert_one(post_data)
print('One post: {0}'.format(result.inserted_id))

One post: 5b5e3f1a69e95585785b4f05


In [17]:
my_stats = db[str(account_id)]
result = my_stats.insert_many(all_stats)
print('Multiple posts: {0}'.format(result.inserted_ids))

Multiple posts: [ObjectId('5b5e3fd469e95585785b4f06'), ObjectId('5b5e3fd469e95585785b4f07'), ObjectId('5b5e3fd469e95585785b4f08'), ObjectId('5b5e3fd469e95585785b4f09'), ObjectId('5b5e3fd469e95585785b4f0a'), ObjectId('5b5e3fd469e95585785b4f0b'), ObjectId('5b5e3fd469e95585785b4f0c'), ObjectId('5b5e3fd469e95585785b4f0d'), ObjectId('5b5e3fd469e95585785b4f0e'), ObjectId('5b5e3fd469e95585785b4f0f'), ObjectId('5b5e3fd469e95585785b4f10'), ObjectId('5b5e3fd469e95585785b4f11'), ObjectId('5b5e3fd469e95585785b4f12'), ObjectId('5b5e3fd469e95585785b4f13'), ObjectId('5b5e3fd469e95585785b4f14'), ObjectId('5b5e3fd469e95585785b4f15'), ObjectId('5b5e3fd469e95585785b4f16'), ObjectId('5b5e3fd469e95585785b4f17'), ObjectId('5b5e3fd469e95585785b4f18'), ObjectId('5b5e3fd469e95585785b4f19'), ObjectId('5b5e3fd469e95585785b4f1a'), ObjectId('5b5e3fd469e95585785b4f1b'), ObjectId('5b5e3fd469e95585785b4f1c'), ObjectId('5b5e3fd469e95585785b4f1d'), ObjectId('5b5e3fd469e95585785b4f1e'), ObjectId('5b5e3fd469e95585785b4f1

In [23]:
client['predictlol'].list_collection_names()

['234507298']

In [8]:
post_1 = {
    'title': 'Python and MongoDB',
    'content': 'PyMongo is fun, you guys',
    'author': 'Scott'
}
post_2 = {
    'title': 'Virtual Environments',
    'content': 'Use virtual environments, you guys',
    'author': 'Scott'
}
post_3 = {
    'title': 'Learning Python',
    'content': 'Learn Python, it is easy',
    'author': 'Bill'
}
new_result = posts.insert_many([post_1, post_2, post_3])
print('Multiple posts: {0}'.format(new_result.inserted_ids))

Multiple posts: [ObjectId('5b5e298c69e9556764a0e36c'), ObjectId('5b5e298c69e9556764a0e36d'), ObjectId('5b5e298c69e9556764a0e36e')]


In [10]:
bills_post = posts.find_one({'author': 'Bill'})
pprint(bills_post)

{'_id': ObjectId('5b5e298c69e9556764a0e36e'),
 'author': 'Bill',
 'content': 'Learn Python, it is easy',
 'title': 'Learning Python'}


In [13]:
scotts_posts = posts.find({'author': 'Scott'})
for post in scotts_posts:
    pprint(post)

{'_id': ObjectId('5b5e298369e9556764a0e36b'),
 'author': 'Scott',
 'content': 'PyMongo is fun, you guys',
 'title': 'Python and MongoDB'}
{'_id': ObjectId('5b5e298c69e9556764a0e36c'),
 'author': 'Scott',
 'content': 'PyMongo is fun, you guys',
 'title': 'Python and MongoDB'}
{'_id': ObjectId('5b5e298c69e9556764a0e36d'),
 'author': 'Scott',
 'content': 'Use virtual environments, you guys',
 'title': 'Virtual Environments'}


In [35]:
# Get 100 ranked matches
ranked_games = db[str(account_id)]
ranked_solo_cursor = ranked_games.find({'queue': 420})

games = 0
ranked_solos = []
while ranked_solo_cursor.alive and games < 100:
    ranked_stats = ranked_solo_cursor.next()
    ranked_solos.append(ranked_stats)
    games += 1
    
game_ids = [game['gameId'] for game in ranked_solos]

In [51]:
ranked_solos

[{'_id': ObjectId('5b5e3fd469e95585785b4f07'),
  'participantId': 8,
  'win': True,
  'item0': 1419,
  'item1': 3022,
  'item2': 3153,
  'item3': 3124,
  'item4': 3047,
  'item5': 3211,
  'item6': 3340,
  'kills': 15,
  'deaths': 7,
  'assists': 3,
  'largestKillingSpree': 8,
  'largestMultiKill': 4,
  'killingSprees': 3,
  'longestTimeSpentLiving': 913,
  'doubleKills': 2,
  'tripleKills': 2,
  'quadraKills': 1,
  'pentaKills': 0,
  'unrealKills': 0,
  'totalDamageDealt': 182613,
  'magicDamageDealt': 4198,
  'physicalDamageDealt': 154716,
  'trueDamageDealt': 23698,
  'largestCriticalStrike': 0,
  'totalDamageDealtToChampions': 20167,
  'magicDamageDealtToChampions': 286,
  'physicalDamageDealtToChampions': 13671,
  'trueDamageDealtToChampions': 6210,
  'totalHeal': 9504,
  'totalUnitsHealed': 1,
  'damageSelfMitigated': 15460,
  'damageDealtToObjectives': 33466,
  'damageDealtToTurrets': 5867,
  'visionScore': 19,
  'timeCCingOthers': 2,
  'totalDamageTaken': 28705,
  'magicalDamage

In [41]:
game_id

[2832141013,
 2832079005,
 2832060727,
 2832031429,
 2832011775,
 2831955077,
 2831951130,
 2831935293,
 2831079042,
 2831073827,
 2830455727,
 2830432417,
 2830407376,
 2830412559,
 2830377229,
 2825665314,
 2825656932,
 2825632541,
 2805389908,
 2805382264,
 2805351128,
 2804671731,
 2804633242,
 2804605205,
 2781872036,
 2776344588,
 2775412317,
 2774402628,
 2747577042,
 2747295013,
 2735356198,
 2735344544,
 2733499415,
 2733485981,
 2732502797,
 2732383753,
 2732139680,
 2732131481,
 2732121083,
 2729908962,
 2729888737,
 2729227636,
 2729199586,
 2728948238,
 2728582737,
 2728527374,
 2728504396,
 2727144192,
 2726106435,
 2726068429,
 2726055241,
 2725964374,
 2725915102,
 2725899275,
 2725901655,
 2725694139,
 2725680164,
 2725645164,
 2724597766,
 2724487664,
 2724427141,
 2724388130,
 2724348648,
 2722907341,
 2722787124,
 2720582998,
 2717417850,
 2717368543,
 2717334874,
 2715332643,
 2711932982,
 2711895770,
 2711876822,
 2710597296,
 2710614498,
 2710560862,
 2710104445,

In [ ]:
import json
# from pymongo import MongoClient
import time
import os, sys, errno

start = time.time()

d = '../data/420/'
try:
    os.makedirs(d)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

def get_all_stats(summoner_name, account_id, api_key, queue='420', limit=100):
    matchlist = []
    start_index = 0

    response = requests.get("https://%s.api.riotgames.com/lol/match/v3/matchlists/by-account/%s?queue=%s&beginIndex=%s&endIndex=%s&api_key=%s" % (region, account_id, queue, start_index, start_index + limit, api_key))
    matchlist_packet = response.json()
    
    all_stats = []
    for match in matchlist:
        time.sleep(2)
        match_data = pl.get_match_by_id(match['gameId'], api_key)
        for participant in match_data['participantIdentities']:
            if participant['player']['summonerName'] == summoner_name:
                participant_id = participant['participantId']

        stats = match_data['participants'][int(participant_id) - 1]['stats']
        for key in match.keys():
            stats[key] = match[key]
        all_stats.append(stats)

        now = time.time()
        delay = (now - start)
        h = int(np.floor(delay / 3600.))
        m = int(np.floor((delay - 3600. * h) / 60.))
        s = (delay - 3600. * h - 60. * m)
        print('Game %s, t = %02d:%02d:%2.3f' % (total, h, m, s))
    return all_stats

# Set up database
# client = MongoClient('mongodb://localhost:27017')
# db = client['predictlol']

# Get all users
users = []
for game_id in game_ids:
    time.sleep(2)
    match = pl.get_match_by_id(game_id, api_key)
    participants = match['participantIdentities']
    for i, participant in enumerate(participants):
        username = participant['player']['summonerName']
        user = pl.get_summoner_ids(username, api_key)
        users.append(user)
users = list(set(users))
print('%s total users' % len(users))

for summoner_name, summoner_id, account_id in users:
    needs_data = False
    try:
        with open('../data/420/%s.json' % account_id, 'r') as fn:
            stats = json.load(fn)
            if len(stats) < 100:
                needs_data = True
    except Exception as e:
        needs_data = True
    if needs_data:
        ranked_stats = get_all_stats(summoner_name, account_id, api_key, queue='420', limit=100)
        with open('../data/420/%s.json' % account_id, 'w') as fn:
            json.dump(ranked_stats, fn)
#     ranked_stats_collection = db[str(account_id)]
#     for stats in ranked_stats:
#         result = ranked_stats_collection.update(stats, stats, upsert=True)
#     print('Finished saving %s\'s data (account number %s)' % (summoner_name, account_id))
    


In [44]:
list(set([1,2,3,3,3,3,3,1]))

[1, 2, 3]

In [45]:
account_id

234507298

In [46]:
response = requests.get("https://%s.api.riotgames.com/lol/match/v3/matchlists/by-account/%s?queue=%s&beginIndex=%s&endIndex=%s&api_key=%s" % ('na1', 234507298, 420, 0, 0 + 100, api_key))
matchlist_packet = response.json()

In [49]:
len(matchlist_packet['matches'])

100

In [50]:
'\''

"'"